In [1]:
import re
import numpy as np
import pandas as pd
from datetime import date, datetime
import warnings

from sklearn.preprocessing import OrdinalEncoder

pd.pandas.set_option('display.max_columns', None)
pd.option_context('display.max_seq_items', None)
pd.options.display.max_rows = 999
warnings.filterwarnings("ignore")

In [2]:
ord_enc = OrdinalEncoder()

In [3]:
stock = "samsung"

In [4]:
nvidia_reddit = pd.read_csv(f"{stock}_full/{stock}FinbertRedditSentiment.csv")
nvidia_reddit.head()

,date,company,score,total_comments,title_neg_sentiment,title_neu_sentiment,title_pos_sentiment,title_compound_sentiment,comment_neg_sentiment,comment_neu_sentiment,comment_pos_sentiment,comment_compound_sentiment,title_textblob_polarity,comment_textblob_polarity,title_textblob_subjectivity,comment_textblob_subjectivity,reputation,groupthink,finbert_positive,finbert_negative,finbert_neutral,finbert_argmax,finbert_comment_positive,finbert_comment_negative,finbert_comment_neutral,finbert_comment_argmax
0,30/9/16,samsung,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1/10/16,samsung,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2/10/16,samsung,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3/10/16,samsung,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4/10/16,samsung,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
nvidia_reddit.shape

(2192, 26)

In [6]:
nvidia_reddit.drop(columns=['title_neg_sentiment',
                            'title_neu_sentiment',
                            'title_pos_sentiment',
                            'comment_neg_sentiment',
                            'comment_neu_sentiment',
                            'comment_pos_sentiment',
                            'finbert_argmax',
                            'finbert_comment_argmax'], inplace=True)

In [7]:
new_col_name = list(nvidia_reddit)[0:2] + ['reddit_' + item for item in list(nvidia_reddit)[2:]]
new_col_name

['date',
 'company',
 'reddit_score',
 'reddit_total_comments',
 'reddit_title_compound_sentiment',
 'reddit_comment_compound_sentiment',
 'reddit_title_textblob_polarity',
 'reddit_comment_textblob_polarity',
 'reddit_title_textblob_subjectivity',
 'reddit_comment_textblob_subjectivity',
 'reddit_reputation',
 'reddit_groupthink',
 'reddit_finbert_positive',
 'reddit_finbert_negative',
 'reddit_finbert_neutral',
 'reddit_finbert_comment_positive',
 'reddit_finbert_comment_negative',
 'reddit_finbert_comment_neutral']

In [8]:
nvidia_reddit.set_axis(new_col_name, axis=1, inplace=True)

In [9]:
nvidia_twitter = pd.read_csv(f"{stock}_full/{stock.title()}_Twitter.csv")
nvidia_twitter.head()

,Date,likeCount,retweetCount,quoteCount,User_friendsCount,User_followersCount,User_statusesCount,likeCount_norm,retweetCount_norm,quoteCount_norm,User_followersCount_norm,GroupThink,Polarity_vader_compound,vader_neg,vader_neu,vader_pos,Reputation,Subjectivity
0,2016-09-30,0.5,0.0,0.0,1945.6,1781.7,83935.6,0.166667,0.0,0.0,0.024306,0.0,-0.14158,0.0855,0.8976,0.0169,0.000101,0.308333
1,2016-10-01,0.3,0.0,0.1,1796.5,4563.9,72932.0,0.150000,0.0,0.1,0.120223,0.0,0.12921,0.0308,0.8894,0.0798,0.000260,0.346578
2,2016-10-02,0.1,0.3,0.0,2476.5,1978.5,110491.7,0.100000,0.3,0.0,0.226144,0.0,0.02992,0.0160,0.9563,0.0277,0.000113,0.278333
3,2016-10-03,0.0,0.0,0.0,300.1,1477.1,24942.3,0.000000,0.0,0.0,0.146528,0.0,0.00583,0.0463,0.9146,0.0391,0.000084,0.434286
4,2016-10-04,0.2,0.0,0.0,1401.6,4274.7,88894.4,0.200000,0.0,0.0,0.158471,0.0,0.05565,0.0397,0.8947,0.0656,0.000243,0.432489


In [10]:
nvidia_twitter.shape

(2192, 18)

In [11]:
nvidia_twitter = nvidia_twitter.set_axis([item.lower() for item in nvidia_twitter.columns.tolist()], axis=1, inplace=False)

In [12]:
nvidia_twitter.drop(columns=['vader_neg',
                             'vader_neu',
                             'vader_pos',
                             'likecount_norm',
                             'retweetcount_norm',
                             'quotecount_norm',
                             'user_followerscount_norm'], inplace=True)

In [13]:
nvidia_new_colname = ['twitter_' + item for item in nvidia_twitter.columns.tolist()[1:]]
nvidia_new_colname.insert(0, nvidia_twitter.columns.tolist()[0])

In [14]:
nvidia_twitter = nvidia_twitter.set_axis(nvidia_new_colname, axis=1, inplace=False)
nvidia_twitter.head()

,date,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity
0,2016-09-30,0.5,0.0,0.0,1945.6,1781.7,83935.6,0.0,-0.14158,0.000101,0.308333
1,2016-10-01,0.3,0.0,0.1,1796.5,4563.9,72932.0,0.0,0.12921,0.000260,0.346578
2,2016-10-02,0.1,0.3,0.0,2476.5,1978.5,110491.7,0.0,0.02992,0.000113,0.278333
3,2016-10-03,0.0,0.0,0.0,300.1,1477.1,24942.3,0.0,0.00583,0.000084,0.434286
4,2016-10-04,0.2,0.0,0.0,1401.6,4274.7,88894.4,0.0,0.05565,0.000243,0.432489


In [15]:
nvidia_news = pd.read_csv(f"{stock}_full/{stock}_news.csv")
nvidia_news.head()

,Date,title_pos,title_pos_var,title_neg,title_neg_var,title_neu,title_neu_var,title_subjectivity,text_pos,text_poss_var,text_neg,text_neg_var,text_neu,text_neu_var,text_subjectivity,art_count
0,2016-11-02,0.009170,0.000000,0.963299,0.000000,0.027531,0.000000,0.000000,0.039564,0.000000,0.937597,0.000000,0.022839,0.000000,0.403955,1
1,2016-11-03,0.765425,0.000000,0.026636,0.000000,0.207939,0.000000,0.454545,0.009096,0.000000,0.965153,0.000000,0.025750,0.000000,0.413581,1
2,2016-11-15,0.080893,0.003585,0.399820,0.151697,0.519287,0.108642,0.000000,0.008335,0.000000,0.955000,0.000000,0.036666,0.000000,0.115096,2
3,2016-11-16,0.075241,0.001245,0.122427,0.011257,0.802332,0.005014,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2
4,2016-11-17,0.696949,0.106237,0.011981,0.000045,0.291069,0.107939,0.275000,0.819730,0.013059,0.030836,0.001032,0.149434,0.017892,0.276759,4


In [16]:
nvidia_news.shape

(1263, 16)

In [17]:
nvidia_news = nvidia_news.set_axis([item.lower() for item in nvidia_news.columns.tolist()], axis=1, inplace=False)

In [18]:
nvidia_new_colname = ['news_' + item for item in nvidia_news.columns.tolist()[1:]]
nvidia_new_colname.insert(0, nvidia_news.columns.tolist()[0])

In [19]:
nvidia_news = nvidia_news.set_axis(nvidia_new_colname, axis=1, inplace=False)
nvidia_news.head()

,date,news_title_pos,news_title_pos_var,news_title_neg,news_title_neg_var,news_title_neu,news_title_neu_var,news_title_subjectivity,news_text_pos,news_text_poss_var,news_text_neg,news_text_neg_var,news_text_neu,news_text_neu_var,news_text_subjectivity,news_art_count
0,2016-11-02,0.009170,0.000000,0.963299,0.000000,0.027531,0.000000,0.000000,0.039564,0.000000,0.937597,0.000000,0.022839,0.000000,0.403955,1
1,2016-11-03,0.765425,0.000000,0.026636,0.000000,0.207939,0.000000,0.454545,0.009096,0.000000,0.965153,0.000000,0.025750,0.000000,0.413581,1
2,2016-11-15,0.080893,0.003585,0.399820,0.151697,0.519287,0.108642,0.000000,0.008335,0.000000,0.955000,0.000000,0.036666,0.000000,0.115096,2
3,2016-11-16,0.075241,0.001245,0.122427,0.011257,0.802332,0.005014,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2
4,2016-11-17,0.696949,0.106237,0.011981,0.000045,0.291069,0.107939,0.275000,0.819730,0.013059,0.030836,0.001032,0.149434,0.017892,0.276759,4


In [20]:
nvidia_news.drop(columns=['news_title_pos_var',
                        'news_title_neg_var',
                        'news_title_neu_var',
                        'news_text_poss_var',
                        'news_text_neg_var',
                        'news_text_neu_var',
                        ], inplace=True)

In [21]:
nvidia_financial = pd.read_csv(f"{stock}_full/{stock.title()}_Financials.csv")
nvidia_financial.head()

,Date,Price,Volume,Revenue,EPS,Heikin_trend,PE_ratio
0,2016-09-30,24.346848,11.7627,41173.492,26.95,downtrend,0.903408
1,2016-10-01,24.346848,11.7627,41173.492,26.95,uptrend,0.903408
2,2016-10-02,24.346848,11.7627,41173.492,26.95,uptrend,0.903408
3,2016-10-03,24.346848,11.7627,41173.492,26.95,uptrend,0.903408
4,2016-10-04,24.554510,13.0531,41173.492,26.95,uptrend,0.911114


In [22]:
nvidia_financial.shape

(2191, 7)

In [23]:
nvidia_financial.drop(columns=["Price"], inplace=True)

In [24]:
nvidia_financial.shape

(2191, 6)

In [25]:
nvidia_financial = nvidia_financial.set_axis([item.lower() for item in nvidia_financial.columns.tolist()], axis=1, inplace=False)

In [26]:
nvidia_new_colname = ['fin_' + item for item in nvidia_financial.columns.tolist()[1:]]
nvidia_new_colname.insert(0, nvidia_financial.columns.tolist()[0])

In [27]:
nvidia_financial = nvidia_financial.set_axis(nvidia_new_colname, axis=1, inplace=False)
nvidia_financial.head()

,date,fin_volume,fin_revenue,fin_eps,fin_heikin_trend,fin_pe_ratio
0,2016-09-30,11.7627,41173.492,26.95,downtrend,0.903408
1,2016-10-01,11.7627,41173.492,26.95,uptrend,0.903408
2,2016-10-02,11.7627,41173.492,26.95,uptrend,0.903408
3,2016-10-03,11.7627,41173.492,26.95,uptrend,0.903408
4,2016-10-04,13.0531,41173.492,26.95,uptrend,0.911114


In [28]:
nvidia_financial_adj = pd.read_csv(f"{stock}_full/{stock.title()}_adj.csv")

In [29]:
nvidia_financial_adj = nvidia_financial_adj.set_axis([item.lower() for item in nvidia_financial_adj.columns.tolist()], axis=1, inplace=False)
nvidia_financial_adj.head()

,date,open,high,low,close,adj_close,volume
0,2016-09-30,28.833076,29.204869,28.920720,29.011819,24.346848,11762700.0
1,2016-10-01,28.833076,29.204869,28.920720,29.011819,24.346848,11762700.0
2,2016-10-02,28.833076,29.204869,28.920720,29.011819,24.346848,11762700.0
3,2016-10-03,28.833076,29.204869,28.920720,29.011819,24.346848,11762700.0
4,2016-10-04,29.226762,29.244399,29.164738,29.259273,24.554510,13053100.0


In [30]:
nvidia_financial_adj.shape

(2191, 7)

In [31]:
nvidia_financial_adj.drop(columns=["volume", "adj_close"], inplace=True)

In [32]:
nvidia_financial_adj.rename(columns=lambda x: re.sub('\W', '_', x), inplace=True)

In [33]:
nvidia_new_colname = ['fin_' + item for item in nvidia_financial_adj.columns.tolist()[1:6]]
nvidia_new_colname.insert(0, nvidia_financial_adj.columns.tolist()[0])

In [34]:
nvidia_new_colname

['date', 'fin_open', 'fin_high', 'fin_low', 'fin_close']

In [35]:
nvidia_financial_adj = nvidia_financial_adj.set_axis(nvidia_new_colname, axis=1, inplace=False)
nvidia_financial_adj.head()

,date,fin_open,fin_high,fin_low,fin_close
0,2016-09-30,28.833076,29.204869,28.920720,29.011819
1,2016-10-01,28.833076,29.204869,28.920720,29.011819
2,2016-10-02,28.833076,29.204869,28.920720,29.011819
3,2016-10-03,28.833076,29.204869,28.920720,29.011819
4,2016-10-04,29.226762,29.244399,29.164738,29.259273


### Merging into one dataset by Date

In [36]:
complete_date = pd.DataFrame(pd.date_range(start='30/9/2016', end='30/9/2022'), columns=['date'])
complete_date.shape

(2192, 1)

In [37]:
nvidia_reddit["date"] = pd.to_datetime(nvidia_reddit["date"])
nvidia_twitter["date"] = pd.to_datetime(nvidia_twitter["date"])
nvidia_news["date"] = pd.to_datetime(nvidia_news["date"])
nvidia_financial["date"] = pd.to_datetime(nvidia_financial["date"])
nvidia_financial_adj["date"] = pd.to_datetime(nvidia_financial_adj["date"])

In [38]:
nvidia_news_missing = complete_date.merge(right=nvidia_news, how="left", on="date")
nvidia_news_missing

,date,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count
0,2016-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2187,2022-09-26,0.866182,0.006758,0.127059,0.0,0.462021,0.022944,0.515036,0.510815,1.0
2188,2022-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2189,2022-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2190,2022-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
nvidia_news = nvidia_news_missing.fillna(method="ffill")
nvidia_news = nvidia_news_missing.fillna(value=0)

In [40]:
nvidia_df = nvidia_reddit.merge(nvidia_twitter, on="date")
nvidia_df["date"] = pd.to_datetime(nvidia_df["date"])
nvidia_df = nvidia_df.merge(nvidia_news, on="date")
nvidia_df["date"] = pd.to_datetime(nvidia_df["date"])
nvidia_df = nvidia_df.merge(nvidia_financial, on="date")
nvidia_df["date"] = pd.to_datetime(nvidia_df["date"])
nvidia_df = nvidia_df.merge(nvidia_financial_adj, on="date")
nvidia_df["date"] = pd.to_datetime(nvidia_df["date"])

In [41]:
nvidia_df.head()

,date,company,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_heikin_trend,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close
0,2016-09-30,samsung,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,1945.6,1781.7,83935.6,0.000000,-0.14158,0.000101,0.308333,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.76270,41173.492,26.95,downtrend,0.903408,28.833076,29.204869,28.920720,29.011819
1,2016-10-10,samsung,10.666667,18,-0.012943,-0.004000,-0.161361,0.076296,0.418961,0.105926,72.00000,1.686534,0.064212,0.327634,0.608154,0.061350,0.223539,0.715112,0.5,0.3,0.0,4008.6,71760.1,116883.2,0.000000,-0.20670,0.004087,0.167500,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,25.29330,41173.492,26.95,uptrend,0.937654,29.583943,30.283200,29.423460,30.111575
2,2016-11-10,samsung,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.42857,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.79535,41173.492,26.95,uptrend,0.888171,28.188988,28.291452,28.260035,28.522502
3,2016-12-10,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.20000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.74393,0.187813,0.0,0.049487,0.450491,0.500022,0.386292,1.0,10.65245,41173.492,26.95,uptrend,0.953805,30.858361,30.615726,30.502514,30.630243
4,2016-10-13,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.20000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.4,0.5,0.0,889.3,1626.6,70759.6,0.000000,0.02671,0.000093,0.472619,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,21.86455,41173.492,26.95,uptrend,0.862849,27.528639,27.778265,27.499176,27.709309


In [42]:
nvidia_df_copy = nvidia_df.copy()

In [43]:
nvidia_df_copy.dropna(inplace=True)

In [44]:
heikin_dummies = pd.get_dummies(nvidia_df_copy["fin_heikin_trend"], prefix="fin_heikin_trend", drop_first=True)

In [45]:
nvidia_df_copy = pd.concat([nvidia_df_copy.drop(['fin_heikin_trend'], axis=1), heikin_dummies], axis=1)

In [46]:
nvidia_df_copy.dtypes

date                                    datetime64[ns]
company                                         object
reddit_score                                   float64
reddit_total_comments                            int64
reddit_title_compound_sentiment                float64
reddit_comment_compound_sentiment              float64
reddit_title_textblob_polarity                 float64
reddit_comment_textblob_polarity               float64
reddit_title_textblob_subjectivity             float64
reddit_comment_textblob_subjectivity           float64
reddit_reputation                              float64
reddit_groupthink                              float64
reddit_finbert_positive                        float64
reddit_finbert_negative                        float64
reddit_finbert_neutral                         float64
reddit_finbert_comment_positive                float64
reddit_finbert_comment_negative                float64
reddit_finbert_comment_neutral                 float64
twitter_li

In [47]:
# nvidia_df_copy["fin_stock_change"] = (nvidia_df_copy["fin_close"] - nvidia_df_copy["fin_open"]) / nvidia_df_copy["fin_open"]
# nvidia_df_copy

In [48]:
# nvidia_df_copy["fin_stock_change_decision"] = nvidia_df_copy["fin_stock_change"].apply(lambda x: 1 if x > 0 else -1)

In [49]:
nvidia_df_copy

,date,company,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend
0,2016-09-30,samsung,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,1945.6,1781.7,83935.6,0.000000,-0.14158,0.000101,0.308333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.762700,41173.492,26.95,0.903408,28.833076,29.204869,28.920720,29.011819,0
1,2016-10-10,samsung,10.666667,18,-0.012943,-0.004000,-0.161361,0.076296,0.418961,0.105926,72.000000,1.686534,0.064212,0.327634,0.608154,0.061350,0.223539,0.715112,0.5,0.3,0.0,4008.6,71760.1,116883.2,0.000000,-0.20670,0.004087,0.167500,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,25.293300,41173.492,26.95,0.937654,29.583943,30.283200,29.423460,30.111575,1
2,2016-11-10,samsung,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.428570,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.795350,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1
3,2016-12-10,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.200000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.743930,0.187813,0.0,0.049487,0.450491,0.500022,0.386292,1.0,10.652450,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1
4,2016-10-13,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.200000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.4,0.5,0.0,889.3,1626.6,70759.6,0.000000,0.02671,0.000093,0.472619,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,21.864550,41173.492,26.95,0.862849,27.528639,27.778265,27.499176,27.709309,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,2022-09-25,samsung,1.000000,2,0.203735,-0.006133,0.118687,0.083333,0.251178,0.063889,0.666667,0.262074,0.271598,0.044139,0.684264,0.059397,0.135708,0.804895,1.9,0.5,0.0,1414.8,2192.1,59508.7,0.000000,0.29491,0.000125,0.345833,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,10.555964,62679.911,1.13,34.125754,38.755823,38.696581,38.669816,38.827065,1
2133,2022-09-26,samsung,1.000000,1,0.144199,0.056623,0.015507,0.068913,0.505159,0.228314,0.500000,0.261340,0.053153,0.249043,0.697805,0.047328,0.439043,0.513629,1.1,0.3,0.0,876.6,534.8,6828.0,0.000000,0.04455,0.000030,0.242500,0.866182,0.006758,0.127059,0.0,0.462021,0.022944,0.515036,0.510815,1.0,15.008449,62679.911,1.13,33.319223,37.768758,37.779249,37.785061,37.909424,0
2134,2022-09-27,samsung,1.000000,1,0.294375,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.317750,0.066649,0.013285,0.920066,0.071170,0.187309,0.741521,0.4,0.3,0.1,675.4,103.7,7284.7,0.286045,0.14339,0.0000

In [50]:
nvidia_df_copy.columns.tolist()

['date',
 'company',
 'reddit_score',
 'reddit_total_comments',
 'reddit_title_compound_sentiment',
 'reddit_comment_compound_sentiment',
 'reddit_title_textblob_polarity',
 'reddit_comment_textblob_polarity',
 'reddit_title_textblob_subjectivity',
 'reddit_comment_textblob_subjectivity',
 'reddit_reputation',
 'reddit_groupthink',
 'reddit_finbert_positive',
 'reddit_finbert_negative',
 'reddit_finbert_neutral',
 'reddit_finbert_comment_positive',
 'reddit_finbert_comment_negative',
 'reddit_finbert_comment_neutral',
 'twitter_likecount',
 'twitter_retweetcount',
 'twitter_quotecount',
 'twitter_user_friendscount',
 'twitter_user_followerscount',
 'twitter_user_statusescount',
 'twitter_groupthink',
 'twitter_polarity_vader_compound',
 'twitter_reputation',
 'twitter_subjectivity',
 'news_title_pos',
 'news_title_neg',
 'news_title_neu',
 'news_title_subjectivity',
 'news_text_pos',
 'news_text_neg',
 'news_text_neu',
 'news_text_subjectivity',
 'news_art_count',
 'fin_volume',
 'fin_

In [51]:
GSPC = pd.read_csv("./GSPC.csv")

In [52]:
GSPC.drop(columns=["Open", "High", "Low", "Close", "Adj.Close", "Volume"], inplace=True)

In [53]:
GSPC.rename(columns={'Date': 'trading_date'}, inplace=True)
GSPC

,trading_date
0,2016-09-29
1,2016-09-30
2,2016-10-03
3,2016-10-04
4,2016-10-05
...,...
1532,2022-10-31
1533,2022-11-01
1534,2022-11-02
1535,2022-11-03


In [54]:
GSPC = list(GSPC.squeeze().ravel())

In [55]:
GSPC

['2016-09-29',
 '2016-09-30',
 '2016-10-03',
 '2016-10-04',
 '2016-10-05',
 '2016-10-06',
 '2016-10-07',
 '2016-10-10',
 '2016-10-11',
 '2016-10-12',
 '2016-10-13',
 '2016-10-14',
 '2016-10-17',
 '2016-10-18',
 '2016-10-19',
 '2016-10-20',
 '2016-10-21',
 '2016-10-24',
 '2016-10-25',
 '2016-10-26',
 '2016-10-27',
 '2016-10-28',
 '2016-10-31',
 '2016-11-01',
 '2016-11-02',
 '2016-11-03',
 '2016-11-04',
 '2016-11-07',
 '2016-11-08',
 '2016-11-09',
 '2016-11-10',
 '2016-11-11',
 '2016-11-14',
 '2016-11-15',
 '2016-11-16',
 '2016-11-17',
 '2016-11-18',
 '2016-11-21',
 '2016-11-22',
 '2016-11-23',
 '2016-11-25',
 '2016-11-28',
 '2016-11-29',
 '2016-11-30',
 '2016-12-01',
 '2016-12-02',
 '2016-12-05',
 '2016-12-06',
 '2016-12-07',
 '2016-12-08',
 '2016-12-09',
 '2016-12-12',
 '2016-12-13',
 '2016-12-14',
 '2016-12-15',
 '2016-12-16',
 '2016-12-19',
 '2016-12-20',
 '2016-12-21',
 '2016-12-22',
 '2016-12-23',
 '2016-12-27',
 '2016-12-28',
 '2016-12-29',
 '2016-12-30',
 '2017-01-03',
 '2017-01-

In [56]:
nvidia_df_copy["date"] = nvidia_df_copy["date"].astype(str)
type(nvidia_df_copy["date"][0])

str

In [57]:
nvidia_df_copy["is_trading_day"] = nvidia_df_copy["date"].apply(lambda x: 1 if x in GSPC else 0)
nvidia_df_copy

,date,company,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend,is_trading_day
0,2016-09-30,samsung,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,1945.6,1781.7,83935.6,0.000000,-0.14158,0.000101,0.308333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.762700,41173.492,26.95,0.903408,28.833076,29.204869,28.920720,29.011819,0,1
1,2016-10-10,samsung,10.666667,18,-0.012943,-0.004000,-0.161361,0.076296,0.418961,0.105926,72.000000,1.686534,0.064212,0.327634,0.608154,0.061350,0.223539,0.715112,0.5,0.3,0.0,4008.6,71760.1,116883.2,0.000000,-0.20670,0.004087,0.167500,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,25.293300,41173.492,26.95,0.937654,29.583943,30.283200,29.423460,30.111575,1,1
2,2016-11-10,samsung,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.428570,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,11.795350,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1,1
3,2016-12-10,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.200000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.743930,0.187813,0.0,0.049487,0.450491,0.500022,0.386292,1.0,10.652450,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1,0
4,2016-10-13,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.200000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.4,0.5,0.0,889.3,1626.6,70759.6,0.000000,0.02671,0.000093,0.472619,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,21.864550,41173.492,26.95,0.862849,27.528639,27.778265,27.499176,27.709309,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,2022-09-25,samsung,1.000000,2,0.203735,-0.006133,0.118687,0.083333,0.251178,0.063889,0.666667,0.262074,0.271598,0.044139,0.684264,0.059397,0.135708,0.804895,1.9,0.5,0.0,1414.8,2192.1,59508.7,0.000000,0.29491,0.000125,0.345833,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,10.555964,62679.911,1.13,34.125754,38.755823,38.696581,38.669816,38.827065,1,0
2133,2022-09-26,samsung,1.000000,1,0.144199,0.056623,0.015507,0.068913,0.505159,0.228314,0.500000,0.261340,0.053153,0.249043,0.697805,0.047328,0.439043,0.513629,1.1,0.3,0.0,876.6,534.8,6828.0,0.000000,0.04455,0.000030,0.242500,0.866182,0.006758,0.127059,0.0,0.462021,0.022944,0.515036,0.510815,1.0,15.008449,62679.911,1.13,33.319223,37.768758,37.779249,37.785061,37.909424,0,1
2134,2022-09-27,samsung,1.000000,1,0.294375,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.317750,0.066649,0.013285,0.920066,0.071170,0.187309,0.741521,0.4,0.3,0.1,675.4,103

In [58]:
nvidia_df_copy["date"] = pd.to_datetime(nvidia_df_copy["date"])
type(nvidia_df_copy["date"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [59]:
filtered_df = nvidia_df_copy.loc[(nvidia_df_copy['date'] >= '2016-10-30')
                     & (nvidia_df_copy['date'] < '2022-09-01')]
filtered_df

,date,company,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend,is_trading_day
2,2016-11-10,samsung,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.42857,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,11.795350,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1,1
3,2016-12-10,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.20000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.743930,0.187813,0.00,0.049487,0.450491,0.500022,0.386292,1.0,10.652450,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1,0
21,2016-10-30,samsung,8.000000,5,0.167625,-0.442350,0.036458,0.020833,0.625000,0.500000,40.00000,1.290595,0.117491,0.028605,0.853904,0.009807,0.927859,0.062334,0.3,0.0,0.0,5447.2,8526.9,40878.7,0.000000,-0.02960,0.000486,0.275000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,10.214200,41173.492,26.95,0.876720,27.572967,28.114057,27.685298,28.154763,1,0
22,2016-10-31,samsung,1.000000,0,0.153100,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.161240,0.025524,0.784041,0.190435,0.071170,0.187309,0.741521,0.1,0.1,0.1,3322.9,3936.7,120063.5,0.100731,0.04187,0.000224,0.244410,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.067200,41173.492,26.95,0.890160,28.181541,28.547790,28.286235,28.586380,1,1
24,2016-11-11,samsung,5.666667,6,0.122732,0.195028,0.046862,0.319444,0.441514,0.384259,11.00000,0.915557,0.048742,0.053226,0.898033,0.145269,0.077887,0.776844,0.4,0.1,0.0,2556.3,4467.4,115426.8,0.000000,0.28526,0.000254,0.400314,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.836800,41173.492,26.95,0.854296,27.270458,27.640401,27.487535,27.434655,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,2022-04-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.3,0.1,0.0,1071.4,778.6,83168.4,0.000000,0.14116,0.000044,0.557323,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,15.453191,64578.913,1.36,40.249800,55.698230,55.477488,55.462253,55.452864,1,0
2115,2022-05-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.2,0.0,0.0,1013.3,706.8,142263.3,0.000000,-0.02834,0.000040,0.326762,0.428177,0.012798,0.559025,0.00,0.092313,0.015573,0.892113,0.370965,1.0,14.356156,64578.913,1.36,38.045428,52.809965,52.591978,52.883546,52.415861,1,1
2116,2022-06-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.8,0.7,0.0,558.6,3235.3

In [60]:
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

,date,company,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend,is_trading_day
0,2016-11-10,samsung,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.42857,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,11.795350,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1,1
1,2016-12-10,samsung,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.20000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.743930,0.187813,0.00,0.049487,0.450491,0.500022,0.386292,1.0,10.652450,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1,0
2,2016-10-30,samsung,8.000000,5,0.167625,-0.442350,0.036458,0.020833,0.625000,0.500000,40.00000,1.290595,0.117491,0.028605,0.853904,0.009807,0.927859,0.062334,0.3,0.0,0.0,5447.2,8526.9,40878.7,0.000000,-0.02960,0.000486,0.275000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,10.214200,41173.492,26.95,0.876720,27.572967,28.114057,27.685298,28.154763,1,0
3,2016-10-31,samsung,1.000000,0,0.153100,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.161240,0.025524,0.784041,0.190435,0.071170,0.187309,0.741521,0.1,0.1,0.1,3322.9,3936.7,120063.5,0.100731,0.04187,0.000224,0.244410,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.067200,41173.492,26.95,0.890160,28.181541,28.547790,28.286235,28.586380,1,1
4,2016-11-11,samsung,5.666667,6,0.122732,0.195028,0.046862,0.319444,0.441514,0.384259,11.00000,0.915557,0.048742,0.053226,0.898033,0.145269,0.077887,0.776844,0.4,0.1,0.0,2556.3,4467.4,115426.8,0.000000,0.28526,0.000254,0.400314,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.836800,41173.492,26.95,0.854296,27.270458,27.640401,27.487535,27.434655,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,2022-04-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.3,0.1,0.0,1071.4,778.6,83168.4,0.000000,0.14116,0.000044,0.557323,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,15.453191,64578.913,1.36,40.249800,55.698230,55.477488,55.462253,55.452864,1,0
2086,2022-05-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.2,0.0,0.0,1013.3,706.8,142263.3,0.000000,-0.02834,0.000040,0.326762,0.428177,0.012798,0.559025,0.00,0.092313,0.015573,0.892113,0.370965,1.0,14.356156,64578.913,1.36,38.045428,52.809965,52.591978,52.883546,52.415861,1,1
2087,2022-06-09,samsung,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.8,0.7,0.0,558.6,3235.3,12

In [61]:
filtered_df.columns.tolist()

['date',
 'company',
 'reddit_score',
 'reddit_total_comments',
 'reddit_title_compound_sentiment',
 'reddit_comment_compound_sentiment',
 'reddit_title_textblob_polarity',
 'reddit_comment_textblob_polarity',
 'reddit_title_textblob_subjectivity',
 'reddit_comment_textblob_subjectivity',
 'reddit_reputation',
 'reddit_groupthink',
 'reddit_finbert_positive',
 'reddit_finbert_negative',
 'reddit_finbert_neutral',
 'reddit_finbert_comment_positive',
 'reddit_finbert_comment_negative',
 'reddit_finbert_comment_neutral',
 'twitter_likecount',
 'twitter_retweetcount',
 'twitter_quotecount',
 'twitter_user_friendscount',
 'twitter_user_followerscount',
 'twitter_user_statusescount',
 'twitter_groupthink',
 'twitter_polarity_vader_compound',
 'twitter_reputation',
 'twitter_subjectivity',
 'news_title_pos',
 'news_title_neg',
 'news_title_neu',
 'news_title_subjectivity',
 'news_text_pos',
 'news_text_neg',
 'news_text_neu',
 'news_text_subjectivity',
 'news_art_count',
 'fin_volume',
 'fin_

In [62]:
col = filtered_df.pop("is_trading_day")
filtered_df.insert(2, col.name, col)
filtered_df

,date,company,is_trading_day,reddit_score,reddit_total_comments,reddit_title_compound_sentiment,reddit_comment_compound_sentiment,reddit_title_textblob_polarity,reddit_comment_textblob_polarity,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,reddit_finbert_positive,reddit_finbert_negative,reddit_finbert_neutral,reddit_finbert_comment_positive,reddit_finbert_comment_negative,reddit_finbert_comment_neutral,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_groupthink,twitter_polarity_vader_compound,twitter_reputation,twitter_subjectivity,news_title_pos,news_title_neg,news_title_neu,news_title_subjectivity,news_text_pos,news_text_neg,news_text_neu,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend
0,2016-11-10,samsung,1,89.428571,367,0.009219,-0.022940,0.214498,0.005966,0.404540,0.163253,29209.42857,14.210910,0.047876,0.435894,0.516230,0.055013,0.120313,0.824674,2.7,5.4,1.4,3087.0,127129.1,391266.9,0.118944,-0.11842,0.007241,0.486667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,11.795350,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1
1,2016-12-10,samsung,0,7.600000,30,0.090770,0.036901,0.000000,-0.017591,0.306667,0.171667,156.20000,1.431295,0.143361,0.201734,0.654905,0.054329,0.205089,0.740582,0.1,0.0,0.0,2064.9,1701.0,176422.6,0.000000,0.01375,0.000097,0.477134,0.068257,0.743930,0.187813,0.00,0.049487,0.450491,0.500022,0.386292,1.0,10.652450,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1
2,2016-10-30,samsung,0,8.000000,5,0.167625,-0.442350,0.036458,0.020833,0.625000,0.500000,40.00000,1.290595,0.117491,0.028605,0.853904,0.009807,0.927859,0.062334,0.3,0.0,0.0,5447.2,8526.9,40878.7,0.000000,-0.02960,0.000486,0.275000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,10.214200,41173.492,26.95,0.876720,27.572967,28.114057,27.685298,28.154763,1
3,2016-10-31,samsung,1,1.000000,0,0.153100,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.161240,0.025524,0.784041,0.190435,0.071170,0.187309,0.741521,0.1,0.1,0.1,3322.9,3936.7,120063.5,0.100731,0.04187,0.000224,0.244410,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.067200,41173.492,26.95,0.890160,28.181541,28.547790,28.286235,28.586380,1
4,2016-11-11,samsung,1,5.666667,6,0.122732,0.195028,0.046862,0.319444,0.441514,0.384259,11.00000,0.915557,0.048742,0.053226,0.898033,0.145269,0.077887,0.776844,0.4,0.1,0.0,2556.3,4467.4,115426.8,0.000000,0.28526,0.000254,0.400314,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,12.836800,41173.492,26.95,0.854296,27.270458,27.640401,27.487535,27.434655,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,2022-04-09,samsung,0,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.3,0.1,0.0,1071.4,778.6,83168.4,0.000000,0.14116,0.000044,0.557323,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,15.453191,64578.913,1.36,40.249800,55.698230,55.477488,55.462253,55.452864,1
2086,2022-05-09,samsung,1,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.2,0.0,0.0,1013.3,706.8,142263.3,0.000000,-0.02834,0.000040,0.326762,0.428177,0.012798,0.559025,0.00,0.092313,0.015573,0.892113,0.370965,1.0,14.356156,64578.913,1.36,38.045428,52.809965,52.591978,52.883546,52.415861,1
2087,2022-06-09,samsung,1,1.000000,0,0.379804,0.000000,0.000000,0.000000,0.125000,0.000000,0.00000,0.258172,0.035345,0.042656,0.922000,0.071170,0.187309,0.741521,0.8,0.7,0.0,558.6,3235.3,

In [63]:
filtered_df["date"] = pd.to_datetime(filtered_df["date"])
filtered_df["year"] = filtered_df["date"].dt.year
filtered_df["month"] = filtered_df["date"].dt.month
filtered_df["day_of_week"] = filtered_df["date"].dt.weekday
filtered_df["quarter"] = filtered_df["date"].dt.quarter
filtered_df["year"]=ord_enc.fit_transform(filtered_df["year"].values.reshape(-1, 1)).flatten()

In [64]:
def argmax_finbert (x):
    x = re.findall(r"neu|neg|pos",x)[0]
    if x == "neu":
        return 0
    elif x == "neg":
        return -1
    else:
        return 1 

In [65]:
def argmax_vader (x):
    if x >= 0.05:
        return 1
    elif x <= -0.05:
        return -1
    else: 
        return 0 

In [66]:
filtered_df["reddit_finbert_argmax"] = filtered_df[['reddit_finbert_positive','reddit_finbert_negative','reddit_finbert_neutral']].idxmax(axis="columns").apply(argmax_finbert)
filtered_df["reddit_finbert_comment_argmax"] = filtered_df[['reddit_finbert_comment_positive','reddit_finbert_comment_negative','reddit_finbert_comment_neutral']].idxmax(axis="columns").apply(argmax_finbert)
filtered_df['reddit_title_compound_sentiment'] = filtered_df['reddit_title_compound_sentiment'].apply(argmax_vader)
filtered_df['reddit_comment_compound_sentiment'] = filtered_df['reddit_comment_compound_sentiment'].apply(argmax_vader)

In [67]:
filtered_df['twitter_polarity_vader_compound'] = filtered_df['twitter_polarity_vader_compound'].apply(argmax_vader)

In [68]:
filtered_df["news_title_finbert_argmax"] = filtered_df[['news_title_pos', 'news_title_neg','news_title_neu']].idxmax(axis="columns").apply(argmax_finbert)
filtered_df["news_text_finbert_argmax"] = filtered_df[[ 'news_text_pos','news_text_neg','news_text_neu']].idxmax(axis="columns").apply(argmax_finbert)

In [69]:
filtered_df.drop(columns=['reddit_finbert_positive',
                        'reddit_finbert_negative',
                        'reddit_finbert_neutral',
                        'reddit_finbert_comment_positive',
                        'reddit_finbert_comment_negative',
                        'reddit_finbert_comment_neutral',
                        'news_title_pos',
                        'news_title_neg',
                        'news_title_neu',
                        'news_text_pos',
                        'news_text_neg',
                        'news_text_neu', 
                        'reddit_title_textblob_polarity',
                        'reddit_comment_textblob_polarity',
                        'company'
                        ], inplace= True)

In [70]:
filtered_df["reddit_title_sentiment"] = filtered_df[['reddit_title_compound_sentiment', 'reddit_finbert_argmax']].mean(axis=1)
filtered_df["reddit_comment_sentiment"] = filtered_df[['reddit_comment_compound_sentiment','reddit_finbert_comment_argmax']].mean(axis=1)

filtered_df.drop(columns=['reddit_title_compound_sentiment', 'reddit_finbert_argmax','reddit_comment_compound_sentiment','reddit_finbert_comment_argmax'], inplace= True)

In [71]:
filtered_df.columns.tolist()

['date',
 'is_trading_day',
 'reddit_score',
 'reddit_total_comments',
 'reddit_title_textblob_subjectivity',
 'reddit_comment_textblob_subjectivity',
 'reddit_reputation',
 'reddit_groupthink',
 'twitter_likecount',
 'twitter_retweetcount',
 'twitter_quotecount',
 'twitter_user_friendscount',
 'twitter_user_followerscount',
 'twitter_user_statusescount',
 'twitter_groupthink',
 'twitter_polarity_vader_compound',
 'twitter_reputation',
 'twitter_subjectivity',
 'news_title_subjectivity',
 'news_text_subjectivity',
 'news_art_count',
 'fin_volume',
 'fin_revenue',
 'fin_eps',
 'fin_pe_ratio',
 'fin_open',
 'fin_high',
 'fin_low',
 'fin_close',
 'fin_heikin_trend_uptrend',
 'year',
 'month',
 'day_of_week',
 'quarter',
 'news_title_finbert_argmax',
 'news_text_finbert_argmax',
 'reddit_title_sentiment',
 'reddit_comment_sentiment']

In [72]:
filtered_df.shape

(2090, 38)

In [73]:
filtered_df = filtered_df[[
 'date',
 'year',
 'month',
 'day_of_week',
 'quarter',
 'is_trading_day',
 'reddit_score',
 'reddit_total_comments',
 'reddit_title_sentiment',
 'reddit_comment_sentiment',
 'reddit_title_textblob_subjectivity',
 'reddit_comment_textblob_subjectivity',
 'reddit_reputation',
 'reddit_groupthink',
 'twitter_likecount',
 'twitter_retweetcount',
 'twitter_quotecount',
 'twitter_user_friendscount',
 'twitter_user_followerscount',
 'twitter_user_statusescount',
 'twitter_polarity_vader_compound',
 'twitter_subjectivity',
 'twitter_reputation',
 'twitter_groupthink',
 'news_title_finbert_argmax',
 'news_text_finbert_argmax',
 'news_title_subjectivity',
 'news_text_subjectivity',
 'news_art_count',
 'fin_volume',
 'fin_revenue',
 'fin_eps',
 'fin_pe_ratio',
 'fin_open',
 'fin_high',
 'fin_low',
 'fin_close',
 'fin_heikin_trend_uptrend',
]]

In [74]:
filtered_df.head()

,date,year,month,day_of_week,quarter,is_trading_day,reddit_score,reddit_total_comments,reddit_title_sentiment,reddit_comment_sentiment,reddit_title_textblob_subjectivity,reddit_comment_textblob_subjectivity,reddit_reputation,reddit_groupthink,twitter_likecount,twitter_retweetcount,twitter_quotecount,twitter_user_friendscount,twitter_user_followerscount,twitter_user_statusescount,twitter_polarity_vader_compound,twitter_subjectivity,twitter_reputation,twitter_groupthink,news_title_finbert_argmax,news_text_finbert_argmax,news_title_subjectivity,news_text_subjectivity,news_art_count,fin_volume,fin_revenue,fin_eps,fin_pe_ratio,fin_open,fin_high,fin_low,fin_close,fin_heikin_trend_uptrend
0,2016-11-10,0.0,11,3,4,1,89.428571,367,0.0,0.0,0.404540,0.163253,29209.42857,14.210910,2.7,5.4,1.4,3087.0,127129.1,391266.9,-1,0.486667,0.007241,0.118944,1,1,0.0,0.000000,0.0,11.79535,41173.492,26.95,0.888171,28.188988,28.291452,28.260035,28.522502,1
1,2016-12-10,0.0,12,5,4,0,7.600000,30,0.5,0.0,0.306667,0.171667,156.20000,1.431295,0.1,0.0,0.0,2064.9,1701.0,176422.6,0,0.477134,0.000097,0.000000,-1,0,0.0,0.386292,1.0,10.65245,41173.492,26.95,0.953805,30.858361,30.615726,30.502514,30.630243,1
2,2016-10-30,0.0,10,6,4,0,8.000000,5,0.5,-1.0,0.625000,0.500000,40.00000,1.290595,0.3,0.0,0.0,5447.2,8526.9,40878.7,0,0.275000,0.000486,0.000000,1,1,0.0,0.000000,0.0,10.21420,41173.492,26.95,0.876720,27.572967,28.114057,27.685298,28.154763,1
3,2016-10-31,0.0,10,0,4,1,1.000000,0,0.0,0.0,0.000000,0.000000,0.00000,0.161240,0.1,0.1,0.1,3322.9,3936.7,120063.5,0,0.244410,0.000224,0.100731,1,1,0.0,0.000000,0.0,12.06720,41173.492,26.95,0.890160,28.181541,28.547790,28.286235,28.586380,1
4,2016-11-11,0.0,11,4,4,1,5.666667,6,0.5,0.5,0.441514,0.384259,11.00000,0.915557,0.4,0.1,0.0,2556.3,4467.4,115426.8,1,0.400314,0.000254,0.000000,1,1,0.0,0.000000,0.0,12.83680,41173.492,26.95,0.854296,27.270458,27.640401,27.487535,27.434655,1


In [75]:
# filtered_df.to_csv("samsung_combined_latest.csv", index=False)